In [1]:
from andreiNet.neural_net import NeuralNetwork, norm_data, one_hot_encode

In [2]:
# Load Iris Dataset
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [3]:
iris = datasets.load_iris()
X = iris.data  
y = iris.target


feature_idxs = [1, 3] # SET FEATURES BY INDEX <------------------

feature_names = ['Sepal Length', 'Sepal Width', 'Petal Length', 'Petal Width']
xlbl, ylbl = feature_names[feature_idxs[0]], feature_names[feature_idxs[1]] 
# We will also split the dataset into training and testing so we can evaluate the kNN classifier
X_trn_, X_test_, y_trn, y_test = train_test_split(X, 
                                                 y, 
                                                 test_size=0.333, 
                                                 random_state=0,
                                                 stratify=y)
X_trn, X_test = X_trn_[:, feature_idxs], X_test_[:, feature_idxs]

print("X_trn.shape = {}, X_test.shape = {}".format(X_trn.shape, X_test.shape))
X_trn_norm, (trn_mean, trn_std) = norm_data(X_trn)
X_test_norm = (X_test - trn_mean) / trn_std



X_trn.shape = (100, 2), X_test.shape = (50, 2)


In [4]:
activation = 'ReLU'
batch_size = 50
random_state = 0
lr = 0.1
n_epochs = 100

nn = NeuralNetwork(hidden=(8,), 
                   init_weights='he_norm', 
                   activation=activation,
                   shuffle=True,
                   batch_size=batch_size,
                   random_state=random_state
                   )
nn.train(X_trn_norm, y_trn, n_epochs=n_epochs, lr=lr)
y_pred_trn = nn.predict(X_trn_norm).argmax(axis=1)
y_pred_test = nn.predict(X_test_norm).argmax(axis=1)

print('trn acc', accuracy_score(y_pred_trn, y_trn))
print('test acc', accuracy_score(y_pred_test, y_test))

epoch 100: final trn loss = 0.34065946969420763
trn acc 0.97
test acc 0.94


In [5]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
import numpy as np

np.random.seed(random_state)

model = Sequential()
model.add(Dense(8, activation=activation.lower(), 
                input_dim=X_trn_norm.shape[1], 
                kernel_initializer='he_normal'))
model.add(Dense(3, activation='softmax',))

sgd = SGD(lr=lr, decay=0.0, momentum=0.00, nesterov=False)

model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.fit(X_trn_norm, one_hot_encode(y_trn, 3),
          epochs=n_epochs,
          batch_size=batch_size)
y_pred_trn = model.predict(X_trn_norm).argmax(axis=1)
y_pred_test = model.predict(X_test_norm).argmax(axis=1)

print('trn acc', accuracy_score(y_pred_trn, y_trn))
print('test acc', accuracy_score(y_pred_test, y_test))

Using TensorFlow backend.
W0901 21:57:39.364915 10440 deprecation_wrapper.py:119] From C:\Users\andre\Anaconda3\envs\py36\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0901 21:57:39.381284 10440 deprecation_wrapper.py:119] From C:\Users\andre\Anaconda3\envs\py36\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0901 21:57:39.383764 10440 deprecation_wrapper.py:119] From C:\Users\andre\Anaconda3\envs\py36\lib\site-packages\keras\backend\tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0901 21:57:39.404101 10440 deprecation_wrapper.py:119] From C:\Users\andre\Anaconda3\envs\py36\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

Epoch 1/100
100/100 [==============================] - 0s 1ms/step - loss: 0.7618 - acc: 0.5800
Epoch 2/100
100/100 [==============================] - 0s 40us/step - loss: 0.6547 - acc: 0.7800
Epoch 3/100
100/100 [==============================] - 0s 30us/step - loss: 0.5978 - acc: 0.7800
Epoch 4/100
100/100 [==============================] - 0s 35us/step - loss: 0.5628 - acc: 0.7800
Epoch 5/100
100/100 [==============================] - 0s 45us/step - loss: 0.5364 - acc: 0.8700
Epoch 6/100
100/100 [==============================] - 0s 45us/step - loss: 0.5138 - acc: 0.8700
Epoch 7/100
100/100 [==============================] - 0s 50us/step - loss: 0.4946 - acc: 0.8700
Epoch 8/100
100/100 [==============================] - 0s 50us/step - loss: 0.4790 - acc: 0.8800
Epoch 9/100
100/100 [==============================] - 0s 30us/step - loss: 0.4631 - acc: 0.8900
Epoch 10/100
100/100 [==============================] - 0s 40us/step - loss: 0.4494 - acc: 0.8900
Epoch 11/100
100/100 [========

100/100 [==============================] - 0s 35us/step - loss: 0.1642 - acc: 0.9700
Epoch 85/100
100/100 [==============================] - 0s 30us/step - loss: 0.1633 - acc: 0.9700
Epoch 86/100
100/100 [==============================] - 0s 30us/step - loss: 0.1627 - acc: 0.9700
Epoch 87/100
100/100 [==============================] - 0s 45us/step - loss: 0.1614 - acc: 0.9600
Epoch 88/100
100/100 [==============================] - 0s 45us/step - loss: 0.1598 - acc: 0.9700
Epoch 89/100
100/100 [==============================] - 0s 30us/step - loss: 0.1584 - acc: 0.9700
Epoch 90/100
100/100 [==============================] - 0s 35us/step - loss: 0.1572 - acc: 0.9700
Epoch 91/100
100/100 [==============================] - 0s 35us/step - loss: 0.1563 - acc: 0.9700
Epoch 92/100
100/100 [==============================] - 0s 35us/step - loss: 0.1552 - acc: 0.9700
Epoch 93/100
100/100 [==============================] - 0s 35us/step - loss: 0.1544 - acc: 0.9700
Epoch 94/100
100/100 [===========